## 데이터베이스 설치
- `mysite/setting.py`파일을 열어보자. 이 파일은 Django 설정을 묘듈 변수로 표현한 Python 모듈이다.
- 76번 라인의 `DATABASES`를 확인해보면 기본적으로 SQLite를 사용하게 되어있음을 확인할 수 있다.
- SQLite는 Python설치시 기본적으로 제공되기에 따로 설치하지 않아도 된다.
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}
```

### 다른 데이터베이스를 사용하려면
- 적절한 데이터베이스 바인딩을 설치하고
- 데이터베이스 연결 설정과 맞게 `DATABASES의 'default'`값을 변경해줍니다.
    - ENGINE
        - 'django.db.backends.sqlite3'
        - 'django.db.backends.postgresql'
        - 'django.db.backends.mysql'
        - 'django.db.backends.oracle'
        - ...
    - NAME
        - 데이터베이스의 이름
- SQLite 를 데이터베이스로 사용하지 않는 경우, `USER`, `PASSWORD`, `HOST` 같은 추가 설정이 반드시 필요합니다. 더 자세한 내용은 DATABASES 문서를 참조해 주세요.


#### ++) settings.py의 INSTALLED_APPS는 뭐지
- 이 파일은 현재 Django 인스턴스에서 활성화된 모든 Django 어플리케이션들의 이름이 담겨 있습니다.
- 앱들은 다수의 프로젝트에서 사용될 수 있고, 다른 프로젝트에서 쉽게 사용될 수 있도록 패키징하여 배포할 수 있습니다.

기본적으로는, INSTALLED_APPS는 Django와 함께 딸려오는 다음의 앱들을 포함합니다.
- django.contrib.admin – 관리용 사이트. 곧 사용하게 될 겁니다.
- django.contrib.auth – 인증 시스템.
- django.contrib.contenttypes – 컨텐츠 타입을 위한 프레임워크.
- django.contrib.sessions – 세션 프레임워크.
- django.contrib.messages – 메세징 프레임워크.
- django.contrib.staticfiles – 정적 파일을 관리하는 프레임워크.

이 어플리케이션들은 일반적인 경우에 사용하기 편리하도록 기본으로 제공됩니다.

## 데이터베이스 테이블을 만들어봅시다.
`$ python manage.py migrate`

#### migrate?
- `INSTALLED_APPS`의 설정을 탐색하여, `mysite/settings.py`의 데이터베이스 설정과 app과 함께 제공되는 database migrations에 따라, 필요한 데이터베이스 테이블을 생성합니다.
- 이를 수행하면 각 migration이 적용되는 메세지가 화면에 출력되는 것을 확인할 수 있습니다.

데이터 마이그레이션: 컴퓨터 시스템에서 데이터베이스나 파일 시스템 등의 데이터를 한 시스템에서 다른 
시스템으로 이전시키는 것을 말합니다.

서버 마이그레이션: 서버를 한 장소에서 다른 장소로 이전시키는 것을 의미합니다.

클라우드 마이그레이션: 기존의 로컬 시스템에서 클라우드 서버로 이전하는 것을 의미합니다.

#### 생성된 데이터베이스를 확인해봅시다
- project(`mysite`)경로로 이동한다
- `python manage.py dbshell` 명령어를 통해 클라이언트에 접속한다
- `.tables` 명령어를 통해 현재 생성된 테이블을 확인해본다
- ctrl+c를 통해 빠져나올 수 있다.

## 모델 만들기
- 모델은 데이터에 대한 하나의 확실한 정보 출처이다.
- 이곳에는 저장 중인 데이터의 필수 필드와 동작이 포함된다.
- Django는 `DRY Principle`를 따른다
- 이는 모델을 한 곳에서 정의하고, 데이터 모델을 자동으로 파생하기 위함이다.

#### 투표앱에서의 모델...
투표앱에서는 `Question`과 `Choice`의 모델을 만들 것이다.<br>
Question에서는 `질문`과 `발행일`을 위한 두 개의 필드를 가지고<br>
Choice에서는 `선택 텍스트`와 `투표 집계`로 두 개의 필드를 가진다.

코드
```python
# polls/models.py
from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
```

- 각각의 모델은 `django.db.models.Model`을 상속하여 정의할 수 있다.
- 각각의 모델은 모델의 데이터베이스 필드를 나타내는 여러 클래스 변수가 있다.
- 각 필드는 필드 클래스의 인스턴스(예: 문자 필드의 경우 CharField, 날짜 시간의 경우 DateTimeField)로 표시됩니다.
    - 정해져있다.
- 각 필드 인스턴스의 이름(예: question_text 또는 pub_date)은 기계에 친숙한 형식의 필드 이름입니다.
    - 이 값은 Python 코드에 사용되고 데이터베이스는 열 이름으로 사용됩니다.
    - 정해져있지 않지만, 파이썬 pep8을 따르는 것이 좋겠지?
- 일부 필드 클래스(CharField 같은 것들)에는 필수 인자가 있다
    - 예를들어 CharField에서는 max_length를 지정해야한다.<br><del>그렇겠지, MySQL에서 VARCHAR로 지정할 수 있는 데이터 길이가 한정되어있으니까 ㅇㅇ</del>
- 필드 클래스에는 다양한 인자가 있을 수 있다. (ex: `default=0`)
- ForeignKey를 이용하여 관계를 정의할 수 있다.

##### 위의 코드를 통해..
- 장고는 앱에 대한 데이터베이스 스키마를 만들 수 있다.
- 그리고 Question 및 Choice 개체에 접근하기 위한 `Python 데이터베이스 엑세스 API`를 만든다.
그러기 위해서는 프로젝트에게 `polls` 앱이 설치되어있음을 알려줘야한다.

### 프로젝트에게 polls 앱이 있다는걸 알려줍시다
- 프로젝트에 polls앱을 포함시키려면 `settings.py`의 `INSTALLED_APP`에 앱의 구성 클래스에 대한 참조를 추가해야한다.
- `PollsConfig`클래스는 `polls/app.py`파일 안에 존재한다.
    - 따라서, 점으로 구분된 경로는 `polls.apps.PollsConfig`가 된다.
- 위의 경로를 `mysite/settins.py`파일의 `INSTALLED_APPS`에 추가하면된다.
```python
# mysite/settings.py

INSTALLED_APPS = [
    'polls.apps.PollsConfig', # 이 부분이 추가된다.
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```

### polls의 모델을 데이터베이스에 적용시킵시다.
`makemigrations` 명령어를 이용합니다.
- `$ python manage.py makemigrations polls`
##### 어떻게 작동되는 것이지?
- `makemigrations`를 실행시킴으로써, 당신이 모델을 변경시킨 사실과 이 변경사항을 migration으로 저장시키고 싶다는 것을 Django에게 알려준다.
- Migration은 Django가 모델의 변경사항을 디스크에 저장하는 방법이다.
- 원하는 경우, `polls/migrations/0001_initial.py`파일로 저장된 새 모델에 대한 migration을 읽어볼 수 있다.
- 또한 Django의 변경점을 수동으로 수정하기를 원할 때를 대비하여 사람이 수정할 수 있도록 설계되어있다.
    - 수정을 위해서는 `migrate`명령어를 사용하면된다.
    - 이 명령어를 알아보기전에 migration이 내부적으로 어떤 SQL문장을 실행하는지 알아보자
    - `sqlmigrate` 명령은 migration 이름을 인수로 받아, 실행하는 SQL 문장을 보여준다.
        - `$ python manage.py sqlmigrate polls 0001`
            ```sql
            BEGIN;
            --
            -- Create model Question
            --
            CREATE TABLE "polls_question" (
                "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
                "question_text" varchar(200) NOT NULL, 
                "pub_date" datetime NOT NULL
            );
            --
            -- Create model Choice
            --
            CREATE TABLE "polls_choice" (
                "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
                "choice_text" varchar(200) NOT NULL, 
                "votes" integer NOT NULL, 
                "question_id" bigint NOT NULL REFERENCES "polls_question" ("id") 
                DEFERRABLE INITIALLY DEFERRED
            );
            CREATE INDEX "polls_choice_question_id_c5b4b260" ON "polls_choice" ("question_id");
            COMMIT;
            ```
        - 사용하는 데이터베이스에 따라서는 출력결과가 다를 수 있다.
        - 테이블이름은 앱의 이름과 모델의 이름이 조합되어 자동으로 생성된다. `(polls_question)`
        - 또한 기본키(ID)가 자동으로 추가된다. `(id)`
            - 이 동작도 재지정할 수 있다.
        - 외래키의 경우 `필드명_id`로 생성된다. `(question_id)`
        - 외래키 관계는 `FOREIGN KEY`에 의해 명시된다.
        - 사용중인 데이터베이스에 맞게 조정되므로, `AUTO_INCREMENT`로 자동 처리 된다.
    - `sqlmigrate` 명령은 데이터베이스에서 마이그레이션을 수행하는 것이 아니라, 화면에 인쇄하여 SQL Django를 확인할 수 있도록 한다.
    - 이는 변경할 것이 있는지 확인하기 위해 사용한다.

## migrate
이제 `migrate`를 실행시켜 데이터베이스에 모델과 관련된 테이블을 생성하자<br>
`$ python manage.py migrate`<br>
`migrate` 명령은 아직 적용되지 않은 마이그레이션을 모두 수집하여 이를 실행하며<br>
이 과정을 통해 모델에서의 변경 사항들과 데이터베이스의 스키마 동기화가 이루어진다.<br>

마이그레이션은 기능이 강력하기에, 프로젝트를 개발할 때처럼 데이터베이스나 테이블에 손대지 않고도<br>
모델의 반복적인 변경을 가능하게 해준다. 또한 동작 중인 데이터베이스를 자료 손실 없이 업그레이드 하는데 최적화되어있다.

### 모델 변경방법
- models.py에서 모델을 변경한다.
- python manage.py makemigrates을 통해 변경사항에 대해 마이그레이션을 만든다.
- python manage.py migrate 명령을 통해 변경사항을 데이터베이스에 적용시킨다.

## 모델을 사용해봅시다
모델을 사용해보기 위해 Python Shell에 접속해봅시다.
```
$ python manage.py shell
```
###### 왜 python이 아닌 위의 명령어로 쉘에 접근하나요?
Django에서 설정한 `settings.py`를 참조하기 위해 위의 명령어를 사용한다.

###### 이제 쉘에 진입했으니 데이터베이스 API를 사용해보자
```python
>>> from polls.models import Choice, Question  # Import the model classes we just wrote.

# No questions are in the system yet.
>>> Question.objects.all()
<QuerySet []>

# Create a new Question.
# Support for time zones is enabled in the default settings file, so
# Django expects a datetime with tzinfo for pub_date. Use timezone.now()
# instead of datetime.datetime.now() and it will do the right thing.
>>> from django.utils import timezone
>>> q = Question(question_text="What's new?", pub_date=timezone.now())

# Save the object into the database. You have to call save() explicitly.
>>> q.save()

# Now it has an ID.
>>> q.id
1

# Access model field values via Python attributes.
>>> q.question_text
"What's new?"
>>> q.pub_date
datetime.datetime(2012, 2, 26, 13, 0, 0, 775217, tzinfo=datetime.timezone.utc)

# Change values by changing the attributes, then calling save().
>>> q.question_text = "What's up?"
>>> q.save()

# objects.all() displays all the questions in the database.
>>> Question.objects.all()
<QuerySet [<Question: Question object (1)>]>
```